Primero vamos a importar pandas que es un modulo de python para manejar bases de datos
Si no lo tienen lo instalan con pip install pandas

In [ ]:
import pandas as pd

Leemos el archivo con los datos del salario y la inflación.
Podemos usar read_csv para leer un archivo csv (separado por comas) o read_excel para leer un archivo de excel


*El SALARIO corresponde al sueldo bruto de un Ayudante de Primera con dedicación semiexclusiva con 10 años de antigüedad y fue extraído de http://agduba.org.ar/. El sueldo en mano es un 23,5% menos.

*El IPC es la variacion mensual (%) del Indice de Precios al Consumidor que elabora el INDEC. La serie completa está en https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-31

In [ ]:
datos = pd.read_csv("salarios_inflacion.csv",parse_dates=["MES"]) #parse_dates dice que columnas tiene que formatear como fechas
#datos = pd.read_excel("salarios_inflacion.xlsx") #Acá no hace falta el parse_dates porque excel ya sabe que son fechas

datos.columns devuelve un Index (una especie de lista) con los nombres de las columnas

datos.dtypes nos dice el tipo de cada columna. queremos tratar al mes como datetime

In [ ]:
#datos.columns
datos.dtypes

datos.head(n) muestra las primeras n filas del dataset

In [ ]:
datos.head(12)

datos.tail(n) muestra las últimas n filas del dataset

In [ ]:
datos.tail(7)

Podemos acceder a una columna como datos.COLUMNA o con datos["COLUMNA"] 

In [ ]:
datos.IPC.tail()
#datos["IPC"].tail()

Para acceder a varias tenemos datos.get([columnas])

In [ ]:
datos.get(["MES","IPC"]).tail()

Dentro de una columna podemos usar indices como en un array

In [ ]:
datos.IPC[0:12]

Para saber el sueldo en mano que percibe un docente hay que descontar el 23,5% de aportes a la jubilación y a la obra social.

In [ ]:
#Creo una nueva columna con el 76,5% del salario burto
datos["SALARIO_MANO"] = datos.SALARIO*0.765
datos.tail()

Y con la notación datos[condicion] obtengo las filas del dataset que cumplen la condicion

In [ ]:
#Ejemplo
datos[datos.SALARIO > 50000]

In [ ]:
# Aca uso .dt que es una forma de acceder a propiedades de columnas de tipo datetime
# en este caso, agarro todo lo que corresponde al año 2022
datos[datos.MES.dt.year == 2022]

Para trabajar con la inflación, necesitamos calcular la inflación acumulada en cierto periodo.

Recordemos que los porcentajes se acumulan multiplicativamente y no aditivamente

In [ ]:
#A esta función le paso una lista de indices de inflación mensual y devuelve la inlfación acumulada en todo el periodo
def inflacion_acumulada(mensuales):
    base = 100
    for mes in mensuales:
        base = base*(1+mes/100)
    return(base-100)

In [ ]:
#Ejemplo. Inflación anual acumuladda con una inflación mensual del 6% 
inflacion_acumulada([6]*12)

In [ ]:
#Así podriamos calcular la inflación anual de 2018
inflacion_acumulada(datos.IPC[0:12])

In [ ]:
# Y así la inflación acumulada en lo que va de 2022
inflacion_acumulada(datos[datos.MES.dt.year == 2022].IPC)

Podríamos comparar la inflación acumulada en el periodo con el aumento del salario

In [ ]:
inflacion_total = inflacion_acumulada(datos.IPC)
aumento_total = (datos.iloc[-1].SALARIO/datos.iloc[0].SALARIO-1)*100

print("Inflación total: ",inflacion_total)
# datos.iloc[i] rescata la fila i-esima
print("Aumento salarial: ",aumento_total)


In [ ]:
# Un poco más lindo usando f-strings

print(f"Inflación total: {inflacion_total:.2f}%")
print(f"Aumento salarial: {aumento_total:.2f}%")

Vamos a hacer algunos gráficos

In [ ]:
import matplotlib.pyplot as plt

Una opción es comparar la evolución del salario con la inflación acumulada.

O sea, por cuanto se multiplico el salario y por cuanto los precios.

El tema acá es que hay que tomar un punto de referencia. Una opción podría ser enero 2018 que es el primer dato que tenemos. 

In [ ]:
# Armamos una lista con el valor relativo del salario tomamdo como 1 al salario en enero 2018

aumento = [] # Empiezo con una lista vacía
for i in datos.index: # recorro todos los meses
    aumento.append((datos.SALARIO[i]/datos.SALARIO[0])) # agrego a la lista el cociente entre el salario del mes con el salario "0"

In [ ]:
# Armamos una lista con la inflación acumulada para cada mes desde enero 2018 (por cuanto se multilpicaron los precios)

acumulada = [1] # Empiezo con una lista con un 1
for i in datos.index[1:]: #no uso el IPC de enero porque es nuestro "punto 0"
    acumulada.append(acumulada[-1]*(1+datos.IPC[i]/100)) # multiciploc la inflación acumulada hasta el mes anterior por el indice de este mes

In [ ]:
# Si quieren se puede agregar estos datos como columnas del dataset
datos["AUMENTO"] = aumento
datos["ACUMULADA"] = acumulada
datos.tail()

In [ ]:
# Graficamos una curva con la evolución del salario y otra con la inflación
plt.plot(datos.MES, aumento,label="salario")
plt.plot(datos.MES, acumulada,label="inflacion")

plt.legend() #Esto hace que aparezca la referencia de que es cada color

plt.title("Salario vs inflación (base enero 2018 = 100)") # Titulo

Hay muchas opciones para hacerlo un poco más claro. Veamos algunas

In [ ]:
# Graficamos una curva con la evolución del salario y otra con la inflación
plt.plot(datos.MES, aumento,label="salario")
plt.plot(datos.MES, acumulada,label="inflacion")

plt.legend() #Esto hace que aparezca la referencia de que es cada color

plt.title("Salario vs inflación (base enero 2018 = 100)") # Titulo

plt.grid(axis="y") # Grilla para el eje y. Prueben sin argumentos 

# Y con esto decimos donde queremos las marquitas del eje x.
# La funcion strftime sirve para formatear datos datetime
# tiene su propia sintaxis en este caso hago mes/año
plt.xticks(datos.MES[::12],datos.MES[::12].dt.strftime("%b/%y")) 

Otra opción es representar la evolución del "salario real"

O sea, cuanto vale (teniendo en cuenta la inlfación) un sueldo hoy en relación a un punto de referencia

In [ ]:
datos["SALARIOREAL"] = datos.AUMENTO/datos.ACUMULADA

plt.plot(datos.MES,datos.SALARIOREAL)

plt.grid(axis="y")
plt.xticks(datos.MES[::12],datos.MES[::12].dt.strftime("%b/%y")) 

plt.title("Evolución del salario real tomando como base enero 2018")

Podríamos tomar la base como la base como 100 para que el gráfico muestre porcentajes

In [ ]:
datos["SALARIOREAL"] = datos.AUMENTO/datos.ACUMULADA*100

plt.plot(datos.MES,datos.SALARIOREAL, color="red")

plt.grid()
plt.xticks(datos.MES[6::12],datos.MES[6::12].dt.strftime("%b/%y")) 

plt.title("Evolución del salario real tomando como base enero 2018")

plt.ylabel("%")

plt.text(datos.iloc[-1].MES,datos.iloc[-1].SALARIOREAL,f"{datos.iloc[-1].SALARIOREAL:.0f}%",color="white",backgroundcolor='red', fontweight='bold')